<a href="https://colab.research.google.com/github/nibaskumar93n-debug/Morphoinformatics/blob/main/NGS_after_trim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get update -qq
!apt-get install -y -qq bwa samtools

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Selecting previously unselected package bwa.
(Reading database ... 121713 files and directories currently installed.)
Preparing to unpack .../bwa_0.7.17-6_amd64.deb ...
Unpacking bwa (0.7.17-6) ...
Selecting previously unselected package libhtscodecs2:amd64.
Preparing to unpack .../libhtscodecs2_1.1.1-3_amd64.deb ...
Unpacking libhtscodecs2:amd64 (1.1.1-3) ...
Selecting previously unselected package libhts3:amd64.
Preparing to unpack .../libhts3_1.13+ds-2build1_amd64.deb ...
Unpacking libhts3:amd64 (1.13+ds-2build1) ...
Selecting previously unselected package samtools.
Preparing to unpack .../samtools_1.13-4_amd64.deb ...
Unpacking samtools (1.13-4) ...
Setting up libhtscodecs2:amd64 (1.1.1-3) ...
Setting up libhts3:amd64 (1.13+ds-2build1) ...
Setting up bwa (0.7.17-6) ...
Setting up samtools (1.13-4

In [3]:
%%bash
# Define the output file name (often ending in .fa or .fasta)
OUTPUT_FILE="GRCh38.primary_assembly.fa.gz"

# Download the compressed FASTA file
echo "Downloading GRCh38 primary assembly..."
wget -O $OUTPUT_FILE http://hgdownload.soe.ucsc.edu/goldenPath/hg38/bigZips/hg38.fa.gz

# Decompress the file
echo "Decompressing the file..."
gunzip $OUTPUT_FILE

# The resulting file will be GRCh38.primary_assembly.fa
echo "Download and decompression complete. File name: GRCh38.primary_assembly.fa"

Decompressing the file...
Download and decompression complete. File name: GRCh38.primary_assembly.fa


--2025-11-27 04:35:58--  http://hgdownload.soe.ucsc.edu/goldenPath/hg38/bigZips/hg38.fa.gz
Resolving hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)... 128.114.119.163
Connecting to hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)|128.114.119.163|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 983659424 (938M) [application/x-gzip]
Saving to: ‘GRCh38.primary_assembly.fa.gz’

     0K .......... .......... .......... .......... ..........  0%  528K 30m18s
    50K .......... .......... .......... .......... ..........  0% 1.03M 22m44s
   100K .......... .......... .......... .......... ..........  0%  260M 15m11s
   150K .......... .......... .......... .......... ..........  0% 1.03M 15m10s
   200K .......... .......... .......... .......... ..........  0%  367M 12m9s
   250K .......... .......... .......... .......... ..........  0%  296M 10m8s
   300K .......... .......... .......... .......... ..........  0%  379M 8m41s
   350K .......... .......... ........

In [5]:
!bwa index /content/GRCh38.primary_assembly.fa

[bwa_index] Pack FASTA... 20.35 sec
[bwa_index] Construct BWT for the packed sequence...
[BWTIncCreate] textLength=6418572210, availableWord=463634060
[BWTIncConstructFromPacked] 10 iterations done. 99999986 characters processed.
[BWTIncConstructFromPacked] 20 iterations done. 199999986 characters processed.
[BWTIncConstructFromPacked] 30 iterations done. 299999986 characters processed.
[BWTIncConstructFromPacked] 40 iterations done. 399999986 characters processed.
[BWTIncConstructFromPacked] 50 iterations done. 499999986 characters processed.
[BWTIncConstructFromPacked] 60 iterations done. 599999986 characters processed.
[BWTIncConstructFromPacked] 70 iterations done. 699999986 characters processed.
[BWTIncConstructFromPacked] 80 iterations done. 799999986 characters processed.
[BWTIncConstructFromPacked] 90 iterations done. 899999986 characters processed.
[BWTIncConstructFromPacked] 100 iterations done. 999999986 characters processed.
[BWTIncConstructFromPacked] 110 iterations done. 

In [8]:
# Mount Google Drive (recommended to save results)
from google.colab import drive
drive.mount('/content/drive')

# Set working directory (modify as needed)
import os
os.chdir('/content/drive/MyDrive/breast_cancer_ngs')  # Change to your folder
!pwd

Mounted at /content/drive
/content/drive/MyDrive/breast_cancer_ngs


In [14]:
# Define your sample information
# MODIFY THESE ACCORDING TO YOUR FILE NAMES
normal_r1 = "SRR28341694_1_paired.fq.gz"
normal_r2 = "SRR28341694_2_paired.fq.gz"

reference = "GRCh38.primary_assembly.fa"
sample_name = "Patient1441"  # Change this to your sample ID

print(f"Processing sample: {sample_name}")
print(f"Normal files: {normal_r1}, {normal_r2}")

Processing sample: Patient1441
Normal files: SRR28341694_1_paired.fq.gz, SRR28341694_2_paired.fq.gz


In [ ]:
# Step 1: Align TUMOR sample with BWA-MEM
import os

# Input directory (Google Drive where your reference and FASTQ files are)
input_dir = "/content/drive/MyDrive/breast_cancer_ngs"

# Output directory (Colab local storage - faster)
output_dir = "/content/output"

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)
reference_path = "/content/GRCh38.primary_assembly.fa"
print(f"Input directory: {input_dir}")
print(f"Output directory: {output_dir}")
normal_r1_path = f"{input_dir}/{normal_r1}"
normal_r2_path = f"{input_dir}/{normal_r2}"
output_bam_normal = f"{output_dir}/{sample_name}_normal.bam"

print("Aligning normal sample...")
!bwa mem -t 2 \
  -R '@RG\tID:{sample_name}_normal\tSM:{sample_name}_normal\tLB:lib1\tPL:ILLUMINA' \
  {reference_path} \
  {normal_r1_path} \
  {normal_r2_path} \
  | samtools view -Sb - > {output_bam_normal}

print("Normal alignment complete!")
!ls -lh {output_bam_normal}


Streaming output truncated to the last 5000 lines.
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (1, 1479)
[M::mem_pestat] mean and std.dev: (325.05, 321.90)
[M::mem_pestat] low and high boundaries for proper pairs: (1, 1933)
[M::mem_pestat] skip orientation FF
[M::mem_pestat] skip orientation RF
[M::mem_pestat] skip orientation RR
[M::mem_process_seqs] Processed 199138 reads in 53.462 CPU sec, 34.256 real sec
[M::process] read 199134 sequences (20000038 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (127, 76648, 68, 94)
[M::mem_pestat] analyzing insert size distribution for orientation FF...
[M::mem_pestat] (25, 50, 75) percentile: (104, 212, 828)
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (1, 2276)
[M::mem_pestat] mean and std.dev: (475.09, 559.98)
[M::mem_pestat] low and high boundaries for proper pairs: (1, 3000)
[M::mem_pestat] analyzing insert size distribution for orientation FR...
[M::mem_pestat] (25,